In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core imports
from core.portfolio.portfolio_manager_v2 import PortfolioManagerV2
from core.multi_asset_loader import load_assets

# Signal imports - using all available strategies
from signals.momentum import MomentumSignal, MomentumSignalV2
from signals.mean_reversion import MeanReversionSignal
from signals.trend_following_long_short import TrendFollowingLongShort, AdaptiveTrendFollowing
from signals.ensemble import AdaptiveEnsemble

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Imports complete")

✅ Imports complete


## 1. Load Historical Data & Benchmark

Load futures data (ES, NQ, GC) and benchmark (SPY), split into in-sample (2010-2024) and out-of-sample (2025-now) periods.

In [2]:
# Load futures data using multi-asset loader
prices = load_assets(
    tickers=['ES', 'NQ', 'GC'],
    start_date='2010-01-01',
    use_yfinance=True  # Fetch recent data if CSV is outdated
)

# Load benchmark (SPY) using yfinance directly
import yfinance as yf
print("\n📊 Loading SPY benchmark data...")
spy_data = yf.download('SPY', start='2010-01-01', progress=False)
benchmark = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
benchmark.index.name = 'Date'

# Split into in-sample and out-of-sample periods
in_sample_end = '2024-12-31'
oos_start = '2025-01-01'

prices_in_sample = {k: v[v.index <= in_sample_end] for k, v in prices.items()}
prices_out_of_sample = {k: v[v.index >= oos_start] for k, v in prices.items()}

benchmark_in_sample = benchmark[benchmark.index <= in_sample_end]
benchmark_out_of_sample = benchmark[benchmark.index >= oos_start]

print(f"\n✅ Loaded {len(prices)} assets + benchmark (SPY)")
print(f"\n📊 IN-SAMPLE PERIOD (2010 - 2024):")
for ticker, df in prices_in_sample.items():
    print(f"  {ticker}: {len(df)} rows | {df.index[0].date()} to {df.index[-1].date()}")
print(f"  SPY: {len(benchmark_in_sample)} rows | {benchmark_in_sample.index[0].date()} to {benchmark_in_sample.index[-1].date()}")

print(f"\n📊 OUT-OF-SAMPLE PERIOD (2025 - Now):")
for ticker, df in prices_out_of_sample.items():
    if len(df) > 0:
        print(f"  {ticker}: {len(df)} rows | {df.index[0].date()} to {df.index[-1].date()}")
    else:
        print(f"  {ticker}: No data available")
if len(benchmark_out_of_sample) > 0:
    print(f"  SPY: {len(benchmark_out_of_sample)} rows | {benchmark_out_of_sample.index[0].date()} to {benchmark_out_of_sample.index[-1].date()}")
else:
    print(f"  SPY: No data available")

# Show sample data
print(f"\nES (S&P 500 E-mini) - Last 5 rows IN-SAMPLE:")
display(prices_in_sample['ES'].tail())



Loading 3 assets: ['ES', 'NQ', 'GC']
  📡 CSV data is 333 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-28)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-28
  📡 CSV data is 333 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-28)
✓ NQ: 6362 rows, 2000-09-18 to 2025-11-28
  📡 CSV data is 333 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-28)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-28
  📡 CSV data is 333 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-28)
✓ NQ: 6362 rows, 2000-09-18 to 2025-11-28
  📡 CSV data is 333 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-28)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-28

Common date range: 2000-09-18 to 2025-11-28
Filtered date range: 2010-01-01 to 2025-11-28

Total unique dates: 4001
  GC:

,Open,High,Low,Close,Volume,Ticker
Date,,,,,,
2024-12-24,6037.75,6099.50,6030.00,6098.00,634201,ES
2024-12-26,6099.25,6107.50,6063.25,6095.25,911486,ES
2024-12-27,6092.00,6095.25,5982.75,6027.00,1641100,ES
2024-12-30,6028.75,6036.25,5918.25,5958.75,1575134,ES
2024-12-31,5955.00,5983.25,5917.25,5935.75,1382187,ES


## 2. Configure Risk Controls

Set up portfolio manager with institutional risk limits.

In [3]:
# Initialize Portfolio Manager with risk controls
pm = PortfolioManagerV2(
    initial_capital=1_000_000,
    risk_per_trade=0.02,          # Risk 2% of capital per trade
    max_position_size=0.25,       # Max 25% of capital per position
    transaction_cost_bps=3.0,     # 3 bps transaction costs
    slippage_bps=2.0,             # 2 bps slippage
    stop_loss_pct=0.10,           # 10% stop loss
    take_profit_pct=0.25,         # 25% take profit
    rebalance_threshold=None,     # No drift-based rebalancing
    rebalance_frequency='never'   # Rebalance only on signals
)

print("✅ Portfolio Manager initialized\n")
print(pm.get_config_summary())

✅ Portfolio Manager initialized


PORTFOLIO MANAGER CONFIGURATION
Initial Capital:       $   1,000,000.00
Risk per Trade:                   2.0%
Max Position Size:               25.0%
Transaction Cost:                  3.0 bps
Slippage:                          2.0 bps
Stop Loss:                         0.1
Take Profit:                      0.25
Rebalance Threshold:              None
Rebalance Frequency:             never



## 3. Define Multi-Strategy Configuration

Set up multiple strategies with different signals for each asset.

In [4]:
# Define multiple strategies for different assets
strategies = [
    {
        'name': 'Adaptive_Ensemble',
        'signal_generator': AdaptiveEnsemble(
            strategies=[
                ('momentum', MomentumSignalV2(lookback=60, entry_threshold=0.02), 0.33),
                ('trend_ls', TrendFollowingLongShort(fast_period=20, slow_period=100), 0.34),
                ("adaptive_trend", AdaptiveTrendFollowing(base_period=60, atr_period=14, vol_lookback=120, base_threshold=0.03), 0.33)
            ],
            method='adaptive',
            adaptive_lookback=60,
            signal_threshold=0.3,
            rebalance_frequency=20
        ),
        'assets': ['ES', 'GC']
    },
    {
        'name': 'TrendFollowing_LS',
        'signal_generator': TrendFollowingLongShort(
            fast_period=20,
            slow_period=100,
            momentum_threshold=0.02,
            volume_multiplier=1.1,
            vol_percentile=0.70
        ),
        'assets': ['NQ', 'GC']
    },
    {
        'name': 'Classic_Momentum',
        'signal_generator': MomentumSignalV2(
            lookback=60,
            entry_threshold=0.02,
            exit_threshold=-0.01
        ),
        'assets': ['GC', 'NQ']
    }
]

print("📊 Strategy Configuration:")
print(f"   Total Strategies: {len(strategies)}")
for s in strategies:
    print(f"   - {s['name']:20s}: {', '.join(s['assets'])}")

📊 Strategy Configuration:
   Total Strategies: 3
   - Adaptive_Ensemble   : ES, GC
   - TrendFollowing_LS   : NQ, GC
   - Classic_Momentum    : GC, NQ


## 4. Generate Signals (IN-SAMPLE)

Generate signals for each strategy-asset combination using IN-SAMPLE data (2010-2024).

In [5]:
# Generate signals for all strategy-asset combinations (IN-SAMPLE)
all_signals_in_sample = {}

for strategy in strategies:
    strategy_name = strategy['name']
    signal_gen = strategy['signal_generator']
    assets = strategy['assets']
    
    print(f"\n🔄 Generating IN-SAMPLE signals for {strategy_name} on {assets}...")
    
    for asset in assets:
        # Generate signal for this asset using IN-SAMPLE data
        df_signal = signal_gen.generate(prices_in_sample[asset].reset_index())
        
        # Store with key: "StrategyName_Asset"
        key = f"{strategy_name}_{asset}"
        all_signals_in_sample[key] = df_signal
        
        # Count signal days
        if 'Position' in df_signal.columns:
            n_long = (df_signal['Position'] == 1).sum()
            n_short = (df_signal['Position'] == -1).sum()
            print(f"   {key}: {n_long} long days, {n_short} short days")

print(f"\n✅ Generated {len(all_signals_in_sample)} IN-SAMPLE signal sets")
print(f"\nSignal keys: {list(all_signals_in_sample.keys())}")



🔄 Generating IN-SAMPLE signals for Adaptive_Ensemble on ['ES', 'GC']...

🔄 Generating IN-SAMPLE signals for TrendFollowing_LS on ['NQ', 'GC']...

🔄 Generating IN-SAMPLE signals for TrendFollowing_LS on ['NQ', 'GC']...

🔄 Generating IN-SAMPLE signals for Classic_Momentum on ['GC', 'NQ']...

✅ Generated 6 IN-SAMPLE signal sets

Signal keys: ['Adaptive_Ensemble_ES', 'Adaptive_Ensemble_GC', 'TrendFollowing_LS_NQ', 'TrendFollowing_LS_GC', 'Classic_Momentum_GC', 'Classic_Momentum_NQ']

🔄 Generating IN-SAMPLE signals for Classic_Momentum on ['GC', 'NQ']...

✅ Generated 6 IN-SAMPLE signal sets

Signal keys: ['Adaptive_Ensemble_ES', 'Adaptive_Ensemble_GC', 'TrendFollowing_LS_NQ', 'TrendFollowing_LS_GC', 'Classic_Momentum_GC', 'Classic_Momentum_NQ']


## 5. Run Backtests (IN-SAMPLE)

Run backtests on IN-SAMPLE data (2010-2024) with proper capital allocation across assets.

**Capital Allocation Architecture:**
- **$1M total capital** split across 3 strategies (35% / 35% / 30%)
- Each strategy runs **once** with **all its assets together**
- Example: Adaptive_Ensemble gets $350k to trade **both ES and GC**
  - The $350k is **shared** between ES and GC positions
  - Portfolio manager ensures total positions don't exceed $350k
  - `max_position_size=0.25` means max $87.5k per asset (25% of $350k)
  
**Why This Matters:**
- ❌ **Wrong**: Running ES and GC separately each with $350k = $700k total (2x leverage!)
- ✅ **Correct**: Running ES and GC together sharing $350k = $350k total (no leverage)

**Position Sizing Rules:**
- `max_position_size=0.25` → Each asset can use max 25% of strategy capital
- `risk_per_trade=0.02` → Risk max 2% of strategy capital per trade
- Portfolio manager handles cross-asset allocation automatically


In [6]:
# IN-SAMPLE BACKTEST: Run each strategy ONCE with ALL its assets
# This ensures capital is shared across assets (no accidental leverage)

# Define capital allocation for each strategy
total_capital = 1_000_000
capital_allocation = {
    'Adaptive_Ensemble': 0.35,  # 35% = $350k shared between ES and GC
    'TrendFollowing_LS': 0.35,  # 35% = $350k shared between NQ and GC
    'Classic_Momentum': 0.30    # 30% = $300k shared between GC and NQ
}

print(f"💰 CAPITAL ALLOCATION (Total: ${total_capital:,.0f})")
print("="*60)
for strategy_name, allocation in capital_allocation.items():
    allocated_capital = total_capital * allocation
    strategy_config = [s for s in strategies if s['name'] == strategy_name][0]
    assets_list = ', '.join(strategy_config['assets'])
    print(f"  {strategy_name:25s}: {allocation:5.1%} = ${allocated_capital:,.0f}")
    print(f"    Assets: {assets_list}")
print("="*60)

# Run IN-SAMPLE backtests
results_in_sample = {}

for strategy in strategies:
    strategy_name = strategy['name']
    strategy_assets = strategy['assets']
    strategy_capital = total_capital * capital_allocation[strategy_name]
    
    print(f"\n{'='*80}")
    print(f"🔄 Running IN-SAMPLE backtest: {strategy_name}")
    print(f"   Period: 2010-2024")
    print(f"   Assets: {', '.join(strategy_assets)}")
    print(f"   Capital: ${strategy_capital:,.0f} (shared across all assets)")
    print(f"{'='*80}")
    
    # Prepare signals dict for ALL assets in this strategy
    signals_dict = {}
    prices_dict = {}
    
    for asset in strategy_assets:
        key = f"{strategy_name}_{asset}"
        df_signal = all_signals_in_sample[key]
        
        # Set Date as index for alignment
        df_signal_indexed = df_signal.set_index('Date')
        
        # Add to signals dict
        signals_dict[asset] = df_signal_indexed[['Signal']]
        
        # Add to prices dict
        prices_dict[asset] = prices_in_sample[asset][['Open', 'High', 'Low', 'Close']]
    
    # Create portfolio manager with allocated capital for this strategy
    pm_strategy = PortfolioManagerV2(
        initial_capital=strategy_capital,
        risk_per_trade=0.02,
        max_position_size=0.25,
        transaction_cost_bps=3.0,
        slippage_bps=2.0,
        stop_loss_pct=0.10,
        take_profit_pct=0.25
    )
    
    # Run backtest with ALL assets
    result = pm_strategy.run_backtest(
        signals=signals_dict,
        prices=prices_dict
    )
    
    # Store result
    results_in_sample[strategy_name] = result
    
    # Print summary
    metrics = result.metrics
    print(f"\n✅ {strategy_name} Complete:")
    print(f"   Initial Capital: ${strategy_capital:,.0f}")
    print(f"   Final Value: ${result.final_equity:,.0f}")
    print(f"   Total Return: {result.total_return:.2%}")
    print(f"   CAGR: {metrics['CAGR']:.2%}")
    print(f"   Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
    print(f"   Max Drawdown: {metrics['Max Drawdown']:.2%}")
    print(f"   Win Rate: {metrics['Win Rate']:.1%}")
    print(f"   Num Trades: {metrics['Total Trades']}")

print(f"\n\n{'='*80}")
print(f"✅ All IN-SAMPLE backtests complete! Ran {len(results_in_sample)} strategies")

# Calculate total portfolio value
total_final_value = sum(result.final_equity for result in results_in_sample.values())
total_return = (total_final_value / total_capital - 1)

print(f"\n💼 IN-SAMPLE PORTFOLIO PERFORMANCE:")
print(f"   Initial Capital: ${total_capital:,.0f}")
print(f"   Final Value: ${total_final_value:,.0f}")
print(f"   Total Return: {total_return:.2%}")
print(f"{'='*80}")


💰 CAPITAL ALLOCATION (Total: $1,000,000)
  Adaptive_Ensemble        : 35.0% = $350,000
    Assets: ES, GC
  TrendFollowing_LS        : 35.0% = $350,000
    Assets: NQ, GC
  Classic_Momentum         : 30.0% = $300,000
    Assets: GC, NQ

🔄 Running IN-SAMPLE backtest: Adaptive_Ensemble
   Period: 2010-2024
   Assets: ES, GC
   Capital: $350,000 (shared across all assets)

✅ Adaptive_Ensemble Complete:
   Initial Capital: $350,000
   Final Value: $578,443
   Total Return: 65.27%
   CAGR: 3.41%
   Sharpe Ratio: 0.32
   Max Drawdown: -8.21%
   Win Rate: 60.9%
   Num Trades: 23

🔄 Running IN-SAMPLE backtest: TrendFollowing_LS
   Period: 2010-2024
   Assets: NQ, GC
   Capital: $350,000 (shared across all assets)

✅ TrendFollowing_LS Complete:
   Initial Capital: $350,000
   Final Value: $512,049
   Total Return: 46.30%
   CAGR: 2.57%
   Sharpe Ratio: 0.18
   Max Drawdown: -5.83%
   Win Rate: 68.8%
   Num Trades: 48

🔄 Running IN-SAMPLE backtest: Classic_Momentum
   Period: 2010-2024
   Assets

## 6. Compare Strategies (IN-SAMPLE)

Side-by-side comparison of all strategies from in-sample period.

In [7]:
# Create comparison DataFrame for IN-SAMPLE strategies
comparison_data_in_sample = {}

for key, result in results_in_sample.items():
    metrics = result.metrics
    comparison_data_in_sample[key] = {
        'Total Return': result.total_return,
        'CAGR': metrics['CAGR'],
        'Sharpe Ratio': metrics['Sharpe Ratio'],
        'Sortino Ratio': metrics['Sortino Ratio'],
        'Max Drawdown': metrics['Max Drawdown'],
        'Calmar Ratio': metrics['Calmar Ratio'],
        'Win Rate': metrics['Win Rate'],
        'Avg Trade': metrics['Avg Trade'],
        'Profit Factor': metrics['Profit Factor'],
        'Total Trades': metrics['Total Trades']
    }

comparison_in_sample = pd.DataFrame(comparison_data_in_sample).T

print("📊 IN-SAMPLE STRATEGY COMPARISON (2010-2024)\n")
print("=" * 100)
print(comparison_in_sample.to_string())

# Find best performers
print(f"\n\n🏆 BEST PERFORMERS (IN-SAMPLE):\n")
print("=" * 60)
print(f"Highest Total Return:  {comparison_in_sample['Total Return'].idxmax():30s} ({comparison_in_sample['Total Return'].max():.2%})")
print(f"Highest Sharpe Ratio:  {comparison_in_sample['Sharpe Ratio'].idxmax():30s} ({comparison_in_sample['Sharpe Ratio'].max():.2f})")
print(f"Lowest Max Drawdown:   {comparison_in_sample['Max Drawdown'].idxmax():30s} ({comparison_in_sample['Max Drawdown'].max():.2%})")
print(f"Highest Win Rate:      {comparison_in_sample['Win Rate'].idxmax():30s} ({comparison_in_sample['Win Rate'].max():.1%})")

# Export comparison
comparison_in_sample.to_csv('../reports/strategy_comparison_in_sample.csv')
print(f"\n✅ Comparison saved to: ../reports/strategy_comparison_in_sample.csv")


📊 IN-SAMPLE STRATEGY COMPARISON (2010-2024)

                   Total Return      CAGR  Sharpe Ratio  Sortino Ratio  Max Drawdown  Calmar Ratio  Win Rate    Avg Trade  Profit Factor  Total Trades
Adaptive_Ensemble      0.652695  0.034125      0.315588       0.398582     -0.082078      0.415767  0.608696  9275.977460       3.411465          23.0
TrendFollowing_LS      0.462998  0.025739      0.180730       0.187482     -0.058314      0.441375  0.687500  3424.923416       3.398847          48.0
Classic_Momentum       0.988461  0.046979      0.523066       0.679998     -0.104885      0.447911  0.645161  9829.775431       3.969343          31.0


🏆 BEST PERFORMERS (IN-SAMPLE):

Highest Total Return:  Classic_Momentum               (98.85%)
Highest Sharpe Ratio:  Classic_Momentum               (0.52)
Lowest Max Drawdown:   TrendFollowing_LS              (-5.83%)
Highest Win Rate:      TrendFollowing_LS              (68.8%)

✅ Comparison saved to: ../reports/strategy_comparison_in_sample.csv

## 7. Generate IN-SAMPLE HTML Report & Risk Dashboard

Generate comprehensive HTML reports with benchmark comparison for in-sample period.

In [ ]:
# Generate IN-SAMPLE HTML Report with Benchmark

from core.reporter import Reporter

# Prepare benchmark equity DataFrame
benchmark_equity_in_sample = benchmark_in_sample[['Close']].copy()
benchmark_equity_in_sample.columns = ['TotalValue']

# Normalize to same starting capital
benchmark_equity_in_sample['TotalValue'] = (benchmark_equity_in_sample['TotalValue'] / benchmark_equity_in_sample['TotalValue'].iloc[0]) * total_capital

# Create reporter
reporter = Reporter(output_dir='../reports/backtest')

# Generate timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
report_path_in_sample = f"../reports/backtest/portfolio_report_in_sample_{timestamp}.html"

# Generate report with benchmark
reporter.generate_multi_strategy_report(
    results=results_in_sample,
    capital_allocation=capital_allocation,
    total_capital=total_capital,
    benchmark_equity=benchmark_equity_in_sample,
    benchmark_name="SPY",
    period_label="In-Sample (2010-2024)",
    save_path=report_path_in_sample,
    auto_open=True
)

print(f"\n{'='*80}")
print(f"✅ IN-SAMPLE HTML Report Generated!")
print(f"{'='*80}")



🔍 DEBUG: Checking equity values...

Total Capital: $1,000,000

Individual Strategy Equity Curves:

Adaptive_Ensemble:
  Initial Capital: $350,000
  Final Equity: $578,443
  First value in equity_curve: $350,000
  Last value in equity_curve: $578,443
  Equity curve shape: (3773, 4)

TrendFollowing_LS:
  Initial Capital: $350,000
  Final Equity: $512,049
  First value in equity_curve: $350,000
  Last value in equity_curve: $512,049
  Equity curve shape: (3773, 4)

Classic_Momentum:
  Initial Capital: $300,000
  Final Equity: $596,538
  First value in equity_curve: $300,000
  Last value in equity_curve: $596,538
  Equity curve shape: (3773, 4)


Expected Portfolio Values:
  Initial: $1,000,000
  Final (sum): $1,687,031

Benchmark Equity:
  First value: $1,000,000
  Last value: $6,812,788
✅ Multi-strategy report saved: ../reports/backtest/portfolio_report_in_sample_20251129_192418.html
🌐 Report opened in browser!

✅ IN-SAMPLE HTML Report Generated!
🌐 Report opened in browser!

✅ IN-SAMPLE

In [9]:
# Generate IN-SAMPLE Risk Dashboard with Benchmark

from core.risk_dashboard import RiskDashboard

# Create risk dashboard
risk_dashboard = RiskDashboard(output_dir='../reports/risk')

# Prepare benchmark data for risk dashboard
benchmark_risk_in_sample = benchmark_equity_in_sample.copy()

# Generate risk dashboard
dashboard_path_in_sample = f"../reports/risk/portfolio_risk_dashboard_in_sample_{timestamp}.html"

risk_dashboard.generate_multi_strategy_risk_dashboard(
    results=results_in_sample,
    capital_allocation=capital_allocation,
    total_capital=total_capital,
    benchmark_data=benchmark_risk_in_sample,
    benchmark_name="SPY",
    save_path=dashboard_path_in_sample,
    auto_open=True
)

print(f"\n{'='*80}")
print(f"✅ IN-SAMPLE Risk Dashboard Generated!")
print(f"{'='*80}")


✅ Risk dashboard saved to: ../reports/risk/portfolio_risk_dashboard_in_sample_20251129_192418.html
🌐 Opening dashboard in browser...

✅ IN-SAMPLE Risk Dashboard Generated!


---

## 8. OUT-OF-SAMPLE TESTING (2025 - Now)

Run the same strategies on out-of-sample data to validate performance.

In [10]:
# Generate OUT-OF-SAMPLE signals for all strategy-asset combinations
all_signals_out_of_sample = {}

# Check if we have out-of-sample data
has_oos_data = any(len(df) > 0 for df in prices_out_of_sample.values())

if not has_oos_data:
    print("⚠️ No out-of-sample data available yet (2025 has just started or data not updated)")
    print("Skipping out-of-sample testing...")
else:
    print(f"📊 Generating OUT-OF-SAMPLE signals (2025 - Now)\n")
    
    for strategy in strategies:
        strategy_name = strategy['name']
        signal_gen = strategy['signal_generator']
        assets = strategy['assets']
        
        print(f"\n🔄 Generating OUT-OF-SAMPLE signals for {strategy_name} on {assets}...")
        
        for asset in assets:
            if len(prices_out_of_sample[asset]) == 0:
                print(f"   ⚠️ No OOS data for {asset}, skipping...")
                continue
                
            # Generate signal for this asset using OUT-OF-SAMPLE data
            df_signal = signal_gen.generate(prices_out_of_sample[asset].reset_index())
            
            # Store with key
            key = f"{strategy_name}_{asset}"
            all_signals_out_of_sample[key] = df_signal
            
            # Count signal days
            if 'Position' in df_signal.columns:
                n_long = (df_signal['Position'] == 1).sum()
                n_short = (df_signal['Position'] == -1).sum()
                print(f"   {key}: {n_long} long days, {n_short} short days")
    
    print(f"\n✅ Generated {len(all_signals_out_of_sample)} OUT-OF-SAMPLE signal sets")


📊 Generating OUT-OF-SAMPLE signals (2025 - Now)


🔄 Generating OUT-OF-SAMPLE signals for Adaptive_Ensemble on ['ES', 'GC']...

🔄 Generating OUT-OF-SAMPLE signals for TrendFollowing_LS on ['NQ', 'GC']...

🔄 Generating OUT-OF-SAMPLE signals for Classic_Momentum on ['GC', 'NQ']...

✅ Generated 6 OUT-OF-SAMPLE signal sets

🔄 Generating OUT-OF-SAMPLE signals for Classic_Momentum on ['GC', 'NQ']...

✅ Generated 6 OUT-OF-SAMPLE signal sets


In [11]:
# Run OUT-OF-SAMPLE backtests

if not has_oos_data:
    print("⏭️ Skipping OUT-OF-SAMPLE backtests (no data available)")
    results_out_of_sample = {}
else:
    results_out_of_sample = {}
    
    print(f"\n{'='*80}")
    print(f"🚀 Running OUT-OF-SAMPLE Backtests (2025 - Now)")
    print(f"{'='*80}\n")
    
    for strategy in strategies:
        strategy_name = strategy['name']
        strategy_assets = strategy['assets']
        strategy_capital = total_capital * capital_allocation[strategy_name]
        
        # Check if all assets have OOS data
        assets_with_data = [asset for asset in strategy_assets if len(prices_out_of_sample[asset]) > 0]
        
        if len(assets_with_data) == 0:
            print(f"⚠️ {strategy_name}: No OOS data for any assets, skipping...")
            continue
        
        print(f"\n{'='*80}")
        print(f"🔄 Running OUT-OF-SAMPLE backtest: {strategy_name}")
        print(f"   Period: 2025 - Now")
        print(f"   Assets: {', '.join(assets_with_data)}")
        print(f"   Capital: ${strategy_capital:,.0f}")
        print(f"{'='*80}")
        
        # Prepare signals and prices for available assets
        signals_dict = {}
        prices_dict = {}
        
        for asset in assets_with_data:
            key = f"{strategy_name}_{asset}"
            if key not in all_signals_out_of_sample:
                continue
                
            df_signal = all_signals_out_of_sample[key]
            df_signal_indexed = df_signal.set_index('Date')
            
            signals_dict[asset] = df_signal_indexed[['Signal']]
            prices_dict[asset] = prices_out_of_sample[asset][['Open', 'High', 'Low', 'Close']]
        
        if len(signals_dict) == 0:
            print(f"   ⚠️ No signals available, skipping...")
            continue
        
        # Create portfolio manager
        pm_strategy = PortfolioManagerV2(
            initial_capital=strategy_capital,
            risk_per_trade=0.02,
            max_position_size=0.25,
            transaction_cost_bps=3.0,
            slippage_bps=2.0,
            stop_loss_pct=0.10,
            take_profit_pct=0.25
        )
        
        # Run backtest
        result = pm_strategy.run_backtest(
            signals=signals_dict,
            prices=prices_dict
        )
        
        results_out_of_sample[strategy_name] = result
        
        # Print summary
        metrics = result.metrics
        print(f"\n✅ {strategy_name} Complete:")
        print(f"   Initial Capital: ${strategy_capital:,.0f}")
        print(f"   Final Value: ${result.final_equity:,.0f}")
        print(f"   Total Return: {result.total_return:.2%}")
        print(f"   CAGR: {metrics['CAGR']:.2%}")
        print(f"   Sharpe Ratio: {metrics['Sharpe Ratio']:.2f}")
        print(f"   Max Drawdown: {metrics['Max Drawdown']:.2%}")
        print(f"   Win Rate: {metrics['Win Rate']:.1%}")
        print(f"   Num Trades: {metrics['Total Trades']}")
    
    if len(results_out_of_sample) > 0:
        total_final_value_oos = sum(result.final_equity for result in results_out_of_sample.values())
        total_return_oos = (total_final_value_oos / total_capital - 1)
        
        print(f"\n\n{'='*80}")
        print(f"✅ All OUT-OF-SAMPLE backtests complete! Ran {len(results_out_of_sample)} strategies")
        print(f"\n💼 OUT-OF-SAMPLE PORTFOLIO PERFORMANCE:")
        print(f"   Initial Capital: ${total_capital:,.0f}")
        print(f"   Final Value: ${total_final_value_oos:,.0f}")
        print(f"   Total Return: {total_return_oos:.2%}")
        print(f"{'='*80}")



🚀 Running OUT-OF-SAMPLE Backtests (2025 - Now)


🔄 Running OUT-OF-SAMPLE backtest: Adaptive_Ensemble
   Period: 2025 - Now
   Assets: ES, GC
   Capital: $350,000

✅ Adaptive_Ensemble Complete:
   Initial Capital: $350,000
   Final Value: $374,737
   Total Return: 7.07%
   CAGR: 7.84%
   Sharpe Ratio: 1.62
   Max Drawdown: -1.90%
   Win Rate: 100.0%
   Num Trades: 1

🔄 Running OUT-OF-SAMPLE backtest: TrendFollowing_LS
   Period: 2025 - Now
   Assets: NQ, GC
   Capital: $350,000

✅ TrendFollowing_LS Complete:
   Initial Capital: $350,000
   Final Value: $350,000
   Total Return: 0.00%
   CAGR: 0.00%
   Sharpe Ratio: 0.00
   Max Drawdown: 0.00%
   Win Rate: 0.0%
   Num Trades: 0

🔄 Running OUT-OF-SAMPLE backtest: Classic_Momentum
   Period: 2025 - Now
   Assets: GC, NQ
   Capital: $300,000

✅ Classic_Momentum Complete:
   Initial Capital: $300,000
   Final Value: $320,466
   Total Return: 6.82%
   CAGR: 7.57%
   Sharpe Ratio: 1.51
   Max Drawdown: -2.03%
   Win Rate: 100.0%
   Num Trades

In [12]:
# Generate OUT-OF-SAMPLE HTML Report & Risk Dashboard

if not has_oos_data or len(results_out_of_sample) == 0:
    print("⏭️ Skipping OUT-OF-SAMPLE reports (no results available)")
else:
    # Prepare benchmark equity
    benchmark_equity_out_of_sample = pd.DataFrame({
        'TotalValue': benchmark_out_of_sample['Close']
    })
    if len(benchmark_equity_out_of_sample) > 0:
        benchmark_equity_out_of_sample['TotalValue'] = (
            benchmark_equity_out_of_sample['TotalValue'] / 
            benchmark_equity_out_of_sample['TotalValue'].iloc[0]
        ) * total_capital
    
    # Generate HTML Report
    report_path_oos = f"../reports/backtest/portfolio_report_out_of_sample_{timestamp}.html"
    
    reporter.generate_multi_strategy_report(
        results=results_out_of_sample,
        capital_allocation=capital_allocation,
        total_capital=total_capital,
        benchmark_equity=benchmark_equity_out_of_sample if len(benchmark_equity_out_of_sample) > 0 else None,
        benchmark_name="SPY",
        period_label="Out-of-Sample (2025 - Now)",
        save_path=report_path_oos,
        auto_open=True
    )
    
    print(f"\n{'='*80}")
    print(f"✅ OUT-OF-SAMPLE HTML Report Generated!")
    print(f"{'='*80}")
    
    # Generate Risk Dashboard
    dashboard_path_oos = f"../reports/risk/portfolio_risk_dashboard_out_of_sample_{timestamp}.html"
    
    risk_dashboard.generate_multi_strategy_risk_dashboard(
        results=results_out_of_sample,
        capital_allocation=capital_allocation,
        total_capital=total_capital,
        benchmark_data=benchmark_equity_out_of_sample if len(benchmark_equity_out_of_sample) > 0 else None,
        benchmark_name="SPY",
        save_path=dashboard_path_oos,
        auto_open=True
    )
    
    print(f"\n{'='*80}")
    print(f"✅ OUT-OF-SAMPLE Risk Dashboard Generated!")
    print(f"{'='*80}")


ValueError: If using all scalar values, you must pass an index

---

## 9. In-Sample vs Out-of-Sample Comparison

Compare performance between training period (2010-2024) and validation period (2025-now).

In [ ]:
# Compare In-Sample vs Out-of-Sample Performance

if len(results_out_of_sample) > 0:
    print("📊 IN-SAMPLE vs OUT-OF-SAMPLE COMPARISON\n")
    print("=" * 120)
    print(f"{'Strategy':<25} | {'IS Return':<12} | {'OOS Return':<12} | {'IS Sharpe':<10} | {'OOS Sharpe':<10} | {'IS MaxDD':<10} | {'OOS MaxDD':<10} | {'Degradation':<15}")
    print("=" * 120)
    
    for strategy_name in results_in_sample.keys():
        is_result = results_in_sample[strategy_name]
        
        if strategy_name in results_out_of_sample:
            oos_result = results_out_of_sample[strategy_name]
            
            is_return = is_result.total_return
            oos_return = oos_result.total_return
            degradation = ((oos_return - is_return) / abs(is_return) * 100) if is_return != 0 else 0
            
            print(f"{strategy_name:<25} | {is_return:>11.2%} | {oos_return:>11.2%} | "
                  f"{is_result.metrics['Sharpe Ratio']:>9.2f} | {oos_result.metrics['Sharpe Ratio']:>9.2f} | "
                  f"{is_result.metrics['Max Drawdown']:>9.2%} | {oos_result.metrics['Max Drawdown']:>9.2%} | "
                  f"{degradation:>13.1f}%")
        else:
            print(f"{strategy_name:<25} | {is_result.total_return:>11.2%} | {'N/A':<12} | "
                  f"{is_result.metrics['Sharpe Ratio']:>9.2f} | {'N/A':<10} | "
                  f"{is_result.metrics['Max Drawdown']:>9.2%} | {'N/A':<10} | {'N/A':<15}")
    
    print("=" * 120)
    
    # Portfolio-level comparison
    is_total = sum(r.final_equity for r in results_in_sample.values())
    is_port_return = (is_total / total_capital - 1)
    
    oos_total = sum(r.final_equity for r in results_out_of_sample.values())
    oos_port_return = (oos_total / total_capital - 1)
    
    port_degradation = ((oos_port_return - is_port_return) / abs(is_port_return) * 100) if is_port_return != 0 else 0
    
    print(f"\n💼 PORTFOLIO SUMMARY:")
    print(f"   In-Sample Return:      {is_port_return:>8.2%}")
    print(f"   Out-of-Sample Return:  {oos_port_return:>8.2%}")
    print(f"   Performance Change:    {port_degradation:>8.1f}%")
    
    if abs(port_degradation) < 20:
        print(f"\n   ✅ GOOD: Performance degradation < 20% (stable strategy)")
    elif abs(port_degradation) < 50:
        print(f"\n   ⚠️  MODERATE: Performance degradation 20-50% (some overfitting)")
    else:
        print(f"\n   ❌ POOR: Performance degradation > 50% (likely overfitted)")
else:
    print("⏭️ Out-of-sample comparison not available (insufficient OOS data)")


## Summary

### What This Notebook Demonstrates:

✅ **Multi-Period Backtesting**:
- In-sample testing (2010-2024) for strategy development
- Out-of-sample validation (2025-now) to detect overfitting
- Benchmark comparison (SPY) for performance attribution
- Walk-forward validation methodology

✅ **Proper Portfolio Architecture**:
- `PortfolioManagerV2` for orchestrating backtests
- Built-in `RiskManager` for position sizing and limits
- `ExecutionEngine` for realistic transaction costs
- `Portfolio` for state tracking
- Multi-asset capital allocation (no accidental leverage)

✅ **Risk Controls**:
- Risk per trade (2% of capital)
- Max position size (25% of capital)
- Stop loss (10%) and take profit (25%)
- Transaction costs (3 bps) and slippage (2 bps)

✅ **Comprehensive Reporting**:
- **HTML Reports**: Portfolio metrics, strategy comparison, equity curves, benchmark comparison, alpha/beta analysis
- **Risk Dashboards**: VaR/CVaR, drawdowns, correlation matrices, covariance analysis, rolling metrics, beta calculations
- **Individual Strategy Charts**: Equity curves for each strategy with Plotly interactive visualizations
- **Performance Attribution**: Strategy vs benchmark analysis with statistical measures

✅ **Statistical Analysis**:
- Beta calculation (strategy volatility vs benchmark)
- Correlation matrices (inter-strategy diversification)
- Covariance matrices (joint variability analysis)
- Alpha generation (excess returns vs benchmark)
- In-sample vs out-of-sample performance comparison

### Key Insights from Reports:

**HTML Report Includes:**
- Portfolio equity curve vs SPY benchmark
- Individual strategy equity curves (interactive Plotly charts)
- Alpha and beta metrics
- Capital allocation visualization
- Strategy comparison tables
- Recent trades per strategy

**Risk Dashboard Includes:**
- VaR/CVaR at 90%, 95%, 99% confidence levels
- Drawdown analysis with underwater charts
- Correlation matrix (strategy diversification)
- Covariance matrix (joint risk)
- Beta vs benchmark (relative volatility)
- Rolling risk metrics (30/60/90-day volatility, Sharpe)
- Individual strategy risk profiles

### Next Steps:

1. **Parameter Optimization**: Grid search on signal parameters using in-sample data
2. **Rolling Window Analysis**: Implement walk-forward optimization
3. **Regime Detection**: Add market regime filters (bull/bear/sideways)
4. **Alternative Benchmarks**: Compare against 60/40 portfolio, CTA index
5. **Live Paper Trading**: Connect to broker API for real-time validation
6. **Monte Carlo Simulation**: Stress test strategies with randomized scenarios
7. **Factor Analysis**: Decompose returns into factor exposures (momentum, value, carry)